In [ ]:
import random
import numpy as np

colors = ['R', 'G', 'B', 'Y']
qualities = ['+', '-']

def generate_chromosome(length):
        return ''.join([random.choice(colors) for _ in range(length)])

def generate_fastq(file_path, num_reads, read_length):
    with open(file_path, 'w') as f:
        for i in range(num_reads):
            read = ''.join([random.choice(colors) for _ in range(read_length)])
            quality = ''.join([random.choice(qualities) for _ in range(read_length)])
            f.write(f"@read_{i}\n{read}\n+\n{quality}\n")

def read_fastq(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    return [{'header': lines[i].strip(), 'sequence': lines[i+1].strip(), 'quality': lines[i+3].strip()} for i in range(0, len(lines), 4)]

def filter_by_quality(reads, min_good_quality):
    def count_good_quality(quality):
        return sum(1 for q in quality if q == '+')

    return [read for read in reads if count_good_quality(read['quality']) >= min_good_quality]

chromosome = generate_chromosome(100)
generate_fastq('lego_reads.fastq', 50, 20)  # 50 reads, each with 20 bricks

reads = read_fastq('lego_reads.fastq')
filtered_reads = filter_by_quality(reads, 10)  # Filter reads with at least 10 good quality bricks


def calculate_score(matrix, i, j, seq1, seq2, match=1, mismatch=-1, gap=-1):
    if seq1[i - 1] == seq2[j - 1]:
        return matrix[i - 1, j - 1] + match
    else:
        return max(
            matrix[i - 1, j - 1] + mismatch,
            matrix[i - 1, j] + gap,
            matrix[i, j - 1] + gap
        )

def align_reads_to_chromosome(chromosome, reads, min_alignment_score=5):
    alignments = []

    for read in reads:
        seq1, seq2 = chromosome, read['sequence']
        m, n = len(seq1), len(seq2)

        # Initialize the similarity matrix
        matrix = np.zeros((m + 1, n + 1))
        max_score, max_i, max_j = 0, 0, 0

        # Calculate scores and find the best local alignment
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                matrix[i, j] = calculate_score(matrix, i, j, seq1, seq2)
                if matrix[i, j] > max_score:
                    max_score, max_i, max_j = matrix[i, j], i, j

        if max_score >= min_alignment_score:
            alignments.append({'header': read['header'], 'start': max_i - n + 1, 'end': max_i, 'score': max_score})

    return alignments

def write_alignments(file_path, alignments):
    with open(file_path, 'w') as f:
        f.write("Read_ID\tStart\tEnd\tScore\n")
        for alignment in alignments:
            f.write(f"{alignment['header']}\t{alignment['start']}\t{alignment['end']}\t{alignment['score']}\n")

# Align reads to the chromosome and write the results to a file
alignments = align_reads_to_chromosome(chromosome, filtered_reads)
write_alignments('lego_alignments.txt', alignments)

def visualize_text_alignment(chromosome, alignments, reads, output_file='alignment_visualization.txt'):
    with open(output_file, 'w') as f:
        genome_view = list(chromosome)
        f.write("Reference genome (chromosome):\n")
        f.write(''.join(genome_view) + '\n')

        for alignment in alignments:
            start, end = alignment['start'] - 1, alignment['end'] - 1
            read = next(r for r in reads if r['header'] == alignment['header'])
            aligned_read = [' '] * len(chromosome)
            aligned_read[start:end + 1] = read['sequence'][0:(end - start + 1)]
            f.write(f"{alignment['header']}:\n")
            f.write(''.join(aligned_read) + '\n')

# Visualize the alignment and write to a file
visualize_text_alignment(chromosome, alignments, filtered_reads)

# This is a toy alignment algorithm based on the Smith-Waterman algorithm
The Smith-Waterman algorithm is a dynamic programming algorithm used for local sequence alignment. It is designed to find the best local alignments between two sequences by assigning scores to matches, mismatches, and gaps. The algorithm creates a scoring matrix that computes the optimal alignment scores for each position in the sequences, and then traces back from the highest score in the matrix to find the optimal local alignment.

Both algorithms use a scoring system to evaluate the quality of the alignment. In our example, we use a simple scoring system that calculates the number of matching bricks, while Smith-Waterman uses a more complex scoring system based on substitution matrices and gap penalties.

The Smith-Waterman algorithm is specifically designed for local alignment, whereas the algorithm used in our example is more focused on a global alignment approach. It aligns the entire read to the reference genome without considering gaps or allowing for partial alignments.

The Smith-Waterman algorithm uses dynamic programming to build a scoring matrix and performs a traceback procedure to identify the optimal alignment. In contrast, our algorithm uses a simpler sliding window approach to find the best matching position in the reference genome, and it doesn't involve building a scoring matrix or performing traceback.